## Header

In [1]:
! pip install pandas
! pip install plotly==4.14.3
! pip install sklearn

In [2]:
# Data handling
import pandas as pd
# Data visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
# ML models

from sklearn import svm
from sklearn import linear_model
from sklearn import neural_network
# ML model accuracy metrics
from sklearn.metrics import mean_squared_error, max_error, mean_absolute_error, median_absolute_error, explained_variance_score
# model persistence
import joblib
# Miscellaneous
from textwrap import shorten
import functools
import time
from numpy import nan, linspace
import numpy as np
colors = (
    '#d54062', '#ffa36c', '#799351', '#0f4c75',
    '#3c2946', '#89c9b8', '#56556e', '#efbbcf',
    '#005086', '#84a9ac','#4f8a8b', '#cedebd', 
    '#00b7c2', '#00bcd4', '#ff5722', '#4f8a8b',
    '#848ccf', '#93b5e1', '#be5683', '#006a71',
    '#5eaaa8', '#5eaaa8', '#557571', '#056676',
    '#0f3460', '#16213e', '#ffaa71', '#fddb3a',
    '#322f3d', '#59405c', '#87556f', '#4b5d67',
    '#b0cac7', '#c3aed6', '#ed6663', '#ffa36c',
    '#158467', '#16213e', '#197163', '#065446',
    '#d9adad', '#776d8a', '#d3c09a', '#797a7e',
)
# xaxis=dict(rangeslider=dict(visible=True))
from IPython.core.display import display, HTML
title_style = '<span style="font-size: 20px;">{0}</span>'
subtitle_style = '<span style="font-size: 16px;">{0}</span>'
tick_style = '<span style="font-size: 10px;">{0}</span>'
custom_style = '<span style="{1}">{0}</span>'
font_size_style = '<span style="font-size:{1}px">{0}</span>'
models = (
        neural_network.MLPRegressor(random_state=100, max_iter=10000, solver='lbfgs', tol=1e-05), 
        svm.SVR(max_iter=80000),
        linear_model.LinearRegression(),
        linear_model.BayesianRidge(),
        linear_model.Ridge(max_iter=10000),
        linear_model.SGDRegressor(random_state=10000),
)

names = ('MLPRegressor', 'SVR', 'LinearRegression','BayesianRidge', 'Ridge', 'SGDRegressor')
alias = ('MLP', 'SVR', 'LR', 'BR', 'R', 'SGD')
alias = dict(zip(names, alias))
images_path = 'MCPR-2021/images/'
city_func = np.median

## Functions

In [4]:
def boxplot_multiindex_df(df, upper_level_index, lower_level_index):
    figure = make_subplots(rows=len(lower_level_index), cols=1, 
                           subplot_titles=tuple(map(lambda s: s.replace('_', ' ').title(),lower_level_index)), 
                           shared_xaxes=True)
    for j, index in enumerate(upper_level_index):
        for i, column in enumerate(lower_level_index):
#             print(index, column)
            figure.add_trace(
                go.Box(
                        name=index,
                        legendgroup=index,
                        marker=dict(color=colors[j]),
                        y=df[index][column], 
                        showlegend=bool(i)
                      ),
                        row=i+1, col=1
            )
    return figure

def dict_of_df_to_multiindex_df(dictionary, axis=1):
    return pd.concat(dictionary.values(), keys=dictionary.keys(), axis=axis)

def split_timeserie_train_test(df, begin, train_size, test_size):
    return (
        (df.iloc[begin + i:train_size + i], 
         df.iloc[train_size + i:train_size + test_size + i]) for i in range(0, len(df) - train_size, test_size)
    )

def transform_df_to_timeserie(df:pd.DataFrame, value:str, column:str, index:str, min_value:str=0) -> pd.DataFrame:
    tmp = df.groupby(column)[value].sum()
    columns = tmp[tmp > min_value].index
    pivot = df.pivot_table(index=index, columns=column, values=value ,aggfunc=sum)[columns]
    return pivot

def plot_multivariate_timeserie(df:pd.DataFrame, normalize:bool=False):
    '''
        Line plot of a dataframe
        Assumes that df has datetime index
        df:           Pandas dataframe where each column is a variable
        normalize:    Normalize by maximum
        
        return plotly figure
    '''
    tmp_df = df/df.max()
    figure = go.Figure()
    for col in tmp_df:
        figure.add_trace(go.Scatter(x=tmp_df.index, y=tmp_df[col], name=col))
    return figure.update_layout(xaxis=dict(rangeslider=dict(visible=True)))

def get_features(df: pd.DataFrame, target):
    target = tuple(target)
    return df.columns[functools.reduce( lambda before, item: before & item,(df.columns != t for t in target))]

def slinding_window(df: pd.DataFrame, width: int):
    return df.shift(width)
    
def multiple_slinding_window(df:pd.DataFrame, periods:iter=[1,2,3]):
#     if not features:
#         features = get_features(df, target)
    return pd.concat((slinding_window(df, i) for i in periods), axis='columns')

def test_model(model, x_train, y_train,x_test, y_test, metrics):
    
    results = dict()
    t1 = time.process_time_ns() # Meassure time before training
    model.fit(x_train, y_train) # Train model inline
    t2 = time.process_time_ns() # Meassure time after training
    predictions = model.predict(x_test) # Predict with the test data
    for metric in metrics:
        results[metric.__name__] = metric(y_test, predictions) # Verify performance
    results['time'] = t2 - t1
    return results

def test_model_progressive(model, x, y, begin, metrics, function=lambda x: x.mean(), step=2):
    '''
        Test a model with a window of n observations and test with the n+k one. After the first test,
        the window is expanded by k observations and it is tested again. The results of the tests are 
        reduced by a given function.
        
        model: (sklear.model) sklearn model that will be trained
        x: (pandas.DataFrame) features 
        y: (pandas.DataFrame o pandas.Series) target
        metrics: (sklearn.metric) Metrics to be meassured
        funcition: (callable) Function that will reduce the results
        begin: (int) The n observation in the x and y
        step: (int) the k value
        
        return: (pd.Series) A serie withe the reduced values of the metrics.
    '''
    step -= 1
    results = {metric.__name__: 0 for metric in metrics}
    end = len(x)
    t1 = time.process_time_ns() # Meassure time before training
    model.fit(x.iloc[:begin], y.iloc[:begin]) # Train model inline
    _range = list(range(1, end - begin, step))
    for i in _range:
        prediction = model.predict(x.iloc[begin + i: begin + i + step])
        for metric in metrics:
            results[metric.__name__] += metric(y.iloc[begin + i: begin + i + step], prediction)/len(_range) # Verify performance
                
        model.fit(x.iloc[begin + i: begin + i + step], y.iloc[begin + i: begin + i + step])
    t2 = time.process_time_ns() # Meassure time before training
    results['time'] = t2 - t1
    return results

def heatmap(x,y,z, colorscale='reds', 
             template='X: %{x}<br>Y: %{y}<br>Z: %{z}'):
    return go.Figure(go.Heatmap(
        z=tmp.abs().T,
        x=tmp.index,
        colorscale='reds',
    #     hoverinfo=tmp.index,
    #     text=tmp.index,
    #     hovertext=tmp.index,
        hovertemplate=template,
        name=''
    ))

## Code

### Dataframe 2

In [5]:
link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/incidencia-cdmx.feather'
# https://github.com/HBaena/forecasting-criminal-incidence/blob/main/rw-wr.feather
df = pd.read_feather(link)
df.head(5)  # 15 municipios con más delitos
keys = df['Municipio'].unique()
df = pd.concat((df.query(f'Municipio=="{city}"') for city in df['Municipio'].unique()),
         keys=keys, axis=0)

In [6]:
value = 'Incidencia'
column = 'Subtipo de delito'
index = 'Date'
min_value = 100

tmp = pd.concat(
    (transform_df_to_timeserie(
        _df, value, column, 
        index, min_value).loc[:'2020-04-01'] for _, _df in df.groupby(level=[0])),
    keys=keys, axis=0
)
# Setting level 1 as datetime
tmp.index = tmp.index.set_levels([
    tmp.index.levels[0],
    pd.to_datetime(tmp.index.levels[1])
]).set_names(['city', 'date'])
# # Sample dataframe by weeks
# tmp = pivot.resample('M').sum()
# # Normalize between 0 and 1
# tmp.head(30)

In [7]:
# plot_multivariate_timeserie(tmp, normalize=True).update_layout(title=title_style.format('Behavior of crimes over time'),
#                                                                xaxis=dict(title='Date'),
#                                                                yaxis=dict(title='Crimes (Normilized)')
#                                                               )

In [8]:
# Normalize
target = 'Homicidio doloso'
features = tmp.columns[tmp.columns != target]

data_max = tmp.groupby(level=0).max().fillna(1)

data = pd.concat((_df/data_max.loc[idx] for idx, _df in tmp.groupby(level=0)), axis=0)

target_col = tmp[target].copy().rename('Target')
target_col = data[target].copy().rename('Target')

## Sliding window

In [9]:
# ---------------
# Slinding window
# ---------------
def test_method(model, data_train_test, metrics, offset):
    cities_dict = dict() 
    for city, _data in data_train_test.groupby(level=0):
        begin = len(data_train_test.loc[city].loc[:'2018'][offset:])
        cities_dict[city] = test_model_progressive(model, 
                            data_train_test.loc[city][features][offset:],
                            data_train_test.loc[city]['Target'][offset:],
                            begin, metrics, 1)
    return cities_dict

def sliding_window_test(data, models, offsets, metrics, replacement=True):
    '''
        Make test over different models using sliding window method
        data:           Data in multivariate format and multi index. 
                        Each index corresponding to a city and each column
                        to a crime
        models:         Iterable of sklear models
        offsets:        Iterable of differet window widths to test
        metrics:        Metrics to meassure
        replacement:    If each iterations will replace data from past ones or not
        
        return a Pandas DataFrame multi index with level [model, offset, city] and 
        columns corresponding of each metric 
    '''
    
    models_dict = dict()
    for model in models:
        offset_dict = dict()
        data_train_test = data[target].rename('Target')
    #     data_train_test = dict()
        for offset in offsets:
            data_train_test = pd.concat(
                                [
                                    data[target].rename('Target') if replacement else \
                                    data_train_test,
                                    data.groupby(level=0).shift(offset)
                                ], axis=1).fillna(0)
#             return data_train_test
            cities_dict = test_method(model, data_train_test, metrics, offset)
            offset_dict[offset] = pd.DataFrame(map(lambda x: pd.Series(x), cities_dict.values()), cities_dict.keys())
        models_dict[model.__class__.__name__] = pd.concat(offset_dict.values(), keys=offset_dict.keys())
    df_multi = pd.concat(models_dict.values(), keys=models_dict.keys())
    df_multi.index = df_multi.index.set_names(['model', 'offset', 'city'])
    return df_multi

def boxplot_multiindex_df(df, column, legend, layout=dict()):
    '''
        Boxplot of a multiindex dataframe (two levels)
        df:         Pandas multiindex DataFrame
        columns:    Columns to plot
        
        return a Plotly figure object (go.Figure)
    '''
    layout = {**dict(showlegend=True,autosize=False), **layout}

    figure = go.Figure()
    for j, _df in enumerate(tmp.groupby(level=0)):
        name, _df = _df
        figure.add_trace(
            go.Box(
                    name=legend[name],
                    marker=dict(color=colors[j]),
                    y=_df[column], 
                  ),
        )
    return figure.update_layout(layout)

def scatter_multiindex_df(df, column,legend, layout=dict()):
    '''
        Scatter of a multiindex dataframe (two levels)
        df:         Pandas multiindex DataFrame
        columns:    Columns to plot
        
        return a Plotly figure object (go.Figure)
    '''
    layout = {**dict(showlegend=True,autosize=False), **layout}
    figure = go.Figure()
    for j, _df in enumerate(tmp.groupby(level=0)):
        name, _df = _df
#         print(_df[column])
        figure.add_trace(
                go.Scatter(
                    x=list(map(lambda x: x[1], _df.index)), 
#                         y=(100*())/() + 1,
                    y=_df[column],
#                     y=(100*(_df[column] - min_.loc[column]))/((max_.loc[column] - min_.loc[column])) + 1, 
                    name=legend[name],
                    marker=dict(color=colors[j]),
                    legendgroup=name,
#                     showlegend=bool(i)
                ))
    return figure.update_layout(layout)

def rolling_stats_window_test(data, models, functions, offsets, metrics, test_method, replacement=True):
    '''
        Make test over different models using sliding window method
        data:           Data in multivariate format and multi index. 
                        Each index corresponding to a city and each column
                        to a crime
        models:         Iterable of sklear models
        offsets:        Iterable of differet window widths to test
        metrics:        Metrics to meassure
        replacement:    If each iterations will replace data from past ones or not
        
        return a Pandas DataFrame multi index with level [model, offset, city] and 
        columns corresponding of each metric 
    '''
    models_dict = dict()
    for model in models:
        functions_dict = dict()
        for name, function in functions.items():
            offset_dict = dict()
            data_train_test = data[target].rename('Target')
            for offset in range(1, 13):
                data_train_test = pd.concat(
                        [
                            data[target].rename('Target') if replacement else \
                            data_train_test,
                            function(data.groupby(level='city').rolling(offset + 1)).reset_index(level=0, drop=True)
                        ], axis=1).fillna(0)
#                 cities_dict = dict()
                cities_dict = test_method(model, data_train_test, metrics, offset)
                offset_dict[offset] = pd.DataFrame(map(lambda x: pd.Series(x), cities_dict.values()), cities_dict.keys())            
            functions_dict[name] = pd.concat(offset_dict.values(), keys=offset_dict.keys())

        models_dict[model.__class__.__name__] = pd.concat(functions_dict.values(), keys=functions_dict.keys())
    df_multi = pd.concat(models_dict.values(), keys=models_dict.keys())
    df_multi.index = df_multi.index.set_names(['model', 'function', 'offset', 'city'])
    return df_multi

### Replacement

In [10]:
# Test sliding window with replacment
tickfont = dict(size = 14,color = 'black',)
margin = dict(
    l=0,
    r=0,
    b=0,
    t=5,
    pad=5
)
fig_width = 500
fig_height = 350 
layout = dict(margin=margin, height=fig_height, width=fig_width, xaxis=dict(tickfont=tickfont,),yaxis=dict(tickfont=tickfont))
replacement = True
metrics = (mean_absolute_error, mean_squared_error)
models = (
        neural_network.MLPRegressor(random_state=100, max_iter=10000, solver='lbfgs', tol=1e-05), 
        svm.SVR(max_iter=80000),
        linear_model.LinearRegression(),
        linear_model.BayesianRidge(),
        linear_model.Ridge(max_iter=10000),
        linear_model.SGDRegressor(random_state=10000),
)
offsets = tuple(range(1, 13))
# df_multi = sliding_window_test(data, models, offsets, metrics, replacement)
link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/sw-r.feather'
df_multi = pd.read_feather(link)
df_multi.set_index(['model', 'offset', 'city'], inplace=True)
# Plot boxplot
tmp = df_multi.groupby(level=[0, 1]).agg(city_func)
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Model'))
).show(render='svg')
# # figure.write_image(f'{images_path}boxplot-sw-error-r.pdf')  # Uncomment locally
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias, layout=layout)
# figure.update_yaxes(type="log")
figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Model')),width=600).show(render='svg')
# # figure.write_image(f'{images_path}scatter-sw-error-r.pdf')  # Uncomment locally
min_sw_r = tmp.groupby(level='model').min()['mean_absolute_error']

In [3]:
import plotly
plotly.__version__

'4.14.3'

In [ ]:
tmp = df_multi.reorder_levels([
    'offset',
    'model',
    'city',
])
tmp = tmp.groupby(level=['offset', 'model']).median()
tmp = tmp[tmp.index.get_level_values('model').isin(['Ridge', 'SVR', 'BayesianRidge'])]
# tmp.loc[1]['mean_absolute_error']
figure = go.Figure()
for i in range(1, 13):
    figure.add_trace(
        go.Box(
            y=tmp.loc[i]['mean_absolute_error'],
            name=f'{i}',
        )
    )
figure.show(render='svg')

### Without replacement

In [11]:
# Test sliding window without replacment
replacement = False
models = (
        neural_network.MLPRegressor(random_state=100, max_iter=10000, solver='lbfgs', tol=1e-05), 
        svm.SVR(max_iter=80000),
        linear_model.LinearRegression(),
        linear_model.BayesianRidge(),
        linear_model.Ridge(max_iter=10000),
        linear_model.SGDRegressor(random_state=10000),
)
# df_multi = sliding_window_test(data, models, offsets, metrics, replacement)
# df_multi = pd.read_csv('sw-wr.csv', index_col=['model', 'offset', 'city'])
link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/sw-wr.feather'
df_multi = pd.read_feather(link)
df_multi.set_index(['model', 'offset', 'city'], inplace=True)
# Plot boxplot
tmp = df_multi.groupby(level=[0, 1]).agg(city_func)
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Model')),
).show(render='svg')
# figure.write_image(f'{images_path}boxplot-sw-error-wr.pdf')
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias, layout=layout)
figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Model')),width=600).show(render='svg')
# figure.write_image(f'{images_path}scatter-sw-error-wr.pdf')
min_sw_wr = tmp.groupby(level='model').min()['mean_absolute_error']

In [ ]:
tmp = df_multi_1.reorder_levels([
    'offset',
    'model',
    'city',
])
tmp = tmp.groupby(level=['offset', 'model']).median() 
# tmp.loc[1]['mean_absolute_error']
figure = go.Figure()
for i in range(1, 13):
    figure.add_trace(
        go.Box(
            y=tmp.loc[i]['mean_absolute_error'],
            name=f'{i}',
        )
    )
figure.show(render='svg')

## Rolling window

### Replacement

In [12]:
# Test rolling window with replacment
replacement = True
metrics = (mean_absolute_error, mean_squared_error)
models = (
        neural_network.MLPRegressor(random_state=100, max_iter=10000), 
        svm.SVR(max_iter=80000),
        linear_model.LinearRegression(),
        linear_model.BayesianRidge(),
        linear_model.Ridge(max_iter=10000),
        linear_model.SGDRegressor(random_state=10000, max_iter=10000),
)
offsets = tuple(range(1, 13))
functions = {
        'mean' : lambda rolling: rolling.mean(),
        'max' : lambda rolling: rolling.max(),
        'min' : lambda rolling: rolling.min(),
        'median' : lambda rolling: rolling.median(),
        'std' : lambda rolling: rolling.std()
}
legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.06,
        xanchor="right",
        x=0.75
)
# df_multi_1 = rolling_stats_window_test(data, models, functions, offsets, metrics, test_method,replacement)
link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/rw-r.feather'
df_multi_1 = pd.read_feather(link)
df_multi_1.set_index(['model', 'function', 'offset', 'city'], inplace=True)
# df_multi_1 = pd.read_csv('rw-r.csv', index_col=)

# Model vs window width
# df_multi_1[df_multi_1 > 1] = nan
tmp = df_multi_1.groupby(level=['model', 'offset']).agg(city_func)
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Model')),
).show(render='svg')
# figure.write_image(f'{images_path}boxplot-rw-model-r.pdf')
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias, layout=layout)

figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Model')),width=600,height=fig_height).show(render='svg')
# figure.write_image(f'{images_path}scatter-rw-model-r.pdf')
figure.data[1]['visible'] = 'legendonly'
figure.data[2]['visible'] = 'legendonly'
figure.data[4]['visible'] = 'legendonly'
figure.update_layout(height=150, showlegend=False,
                      xaxis=dict(visible=True,title='', tickfont=dict(size=8)),
                      yaxis=dict(tickfont=dict(size=8)),
                      )
# figure.write_image(f'{images_path}scatter-rw-best-r.pdf')

# Functions vs window width
alias_f = dict(zip(functions.keys(), map(lambda s:s.capitalize(),functions.keys())))
tmp = df_multi_1.groupby(level=['function', 'offset']).agg(city_func)
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias_f, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Function')),
).show(render='svg')
# figure.write_image(f'{images_path}boxplot-rw-function-r.pdf')
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias_f, layout=layout)

figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Function')),width=600,height=fig_height).show(render='svg')
# figure.write_image(f'{images_path}scatter-rw-function-r.pdf')

tmp = df_multi_1.groupby(level=[0,1, 2]).agg(city_func)

j=0
figure = make_subplots(rows=3, cols=2, shared_xaxes=True, subplot_titles=df_multi_1.index.levels[0],
                      horizontal_spacing=0.07,
                      vertical_spacing=0.06,
                      )
for model, _df in tmp.groupby(level=0):
    i = 0
#     print(model)
    for function, __df in _df.groupby(level=1):
        figure.add_trace(
                go.Scatter(
                    x=list(map(lambda x:x[2], __df.index)),
                    y=__df['mean_absolute_error'],
                    marker=dict(color=colors[i]),
                    name=function,
                    legendgroup=function,
                    showlegend=bool(j == 0)
                ), row=j%3 + 1, col=j%2 + 1
        )
        i+=1
    j+=1
    figure.update_xaxes(nticks=12, row=j%3 + 1, col=j%2 + 1)
    figure.update_xaxes(nticks=12, row=j%3 + 1, col=j%2 + 1)
# figure.update_xaxes(visible=False, row=1, col=1)

figure.update_xaxes(title='Window width', row=3, col=2)
figure.update_xaxes(title='Window width', row=3, col=1)
figure.update_layout(margin=margin, legend=legend).show(render='svg')

# figure.write_image(f'{images_path}scatter-rw-fig-r.pdf')
#         print(f'{images_path}scatter-rw-fig-{j}-r.pdf')
#     'MCPR-2021/images/sca'
tmp = df_multi_1.groupby(level=['model', 'offset']).agg(city_func)
min_rw_r = tmp.groupby(level='model').min()['mean_absolute_error']

### Withput replacemenet

In [13]:
# Test rolling window without replacment
replacement = False
metrics = (mean_absolute_error, mean_squared_error)
models = (
        neural_network.MLPRegressor(random_state=100, max_iter=10000), 
        svm.SVR(max_iter=80000),
        linear_model.LinearRegression(),
        linear_model.BayesianRidge(),
        linear_model.Ridge(max_iter=10000),
        linear_model.SGDRegressor(random_state=10000, max_iter=10000),
)
offsets = tuple(range(1, 13))
legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.06,
        xanchor="right",
        x=0.75
)

# df_multi_1 = rolling_stats_window_test(data, models, functions, offsets, metrics, test_method,replacement)
link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/rw-wr.feather'
df_multi_1 = pd.read_feather(link)
df_multi_1.set_index(['model', 'function', 'offset', 'city'], inplace=True)
# df_multi_1 = pd.read_csv('rw-wr.csv', index_col=['model', 'function', 'offset', 'city'])

# Model vs window width
# df_multi_1[df_multi_1 > 1] = nan
tmp = df_multi_1.groupby(level=['model', 'offset']).agg(city_func)
min_rw_wr = tmp.groupby(level='model').min()['mean_absolute_error']
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Model')),
).show(render='svg')
# figure.write_image(f'{images_path}boxplot-rw-model-wr.pdf')
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias, layout=layout)
figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Model')),width=600,height=fig_height).show(render='svg')
# figure.write_image(f'{images_path}scatter-rw-model-wr.pdf')

# Functions vs window width
alias_f = dict(zip(functions.keys(), map(lambda s:s.capitalize(),functions.keys())))
tmp = df_multi_1.groupby(level=['function', 'offset']).agg(city_func)
figure = boxplot_multiindex_df(tmp, 'mean_absolute_error', alias_f, layout=dict(**layout, **dict(showlegend=False)))
figure.update_xaxes(title=subtitle_style.format('Window width'))
figure.update_layout(
                    width=400,
                    legend=dict(title=subtitle_style.format('Function')),
).show(render='svg')
# figure.write_image(f'{images_path}boxplot-rw-function-wr.pdf')
# Plot scatterplot
figure = scatter_multiindex_df(tmp, 'mean_absolute_error', alias_f, layout=layout)
figure.update_xaxes(title='Window width', nticks=12
                   ).update_xaxes(title=subtitle_style.format('Window width')).update_layout(
                    legend=dict(title=subtitle_style.format('Function')),width=600,height=fig_height).show(render='svg')
# figure.write_image(f'{images_path}scatter-rw-function-wr.pdf')

tmp = df_multi_1.groupby(level=[0,1, 2]).agg(city_func)

j=0
figure = make_subplots(rows=3, cols=2, shared_xaxes=True, subplot_titles=df_multi_1.index.levels[0],
                      horizontal_spacing=0.07,
                      vertical_spacing=0.06,
                      )
for model, _df in tmp.groupby(level=0):
    i = 0
    for function, __df in _df.groupby(level=1):
        figure.add_trace(
                go.Scatter(
                    x=list(map(lambda x:x[2], __df.index)),
                    y=__df['mean_absolute_error'],
                    marker=dict(color=colors[i]),
                    name=function,
                    legendgroup=function,
                    showlegend=bool(j == 0)
                ), row=j%3 + 1, col=j%2 + 1
        )
        i+=1
    j+=1
    figure.update_xaxes(nticks=12, row=j%3 + 1, col=j%2 + 1)
    figure.update_xaxes(nticks=12, row=j%3 + 1, col=j%2 + 1)
# figure.update_xaxes(visible=False, row=1, col=1)

figure.update_xaxes(title='Window width', row=3, col=2)
figure.update_xaxes(title='Window width', row=3, col=1)
figure.update_layout(margin=margin, legend=legend).show(render='svg')
# figure.update_layout(margin=margin, height=fig_height - 200).show(render='svg').update_layout(
#                     legend=legend,
#                     showlegend=bool(j == 2),
#                     xaxis=dict(visible=bool(j == 6), title='Window width' , nticks=12, showticklabels=bool(j == 6)),
#                     yaxis=dict(title='Absolute error'),
# ).show(render='svg').update_xaxes(visible=bool(j == 6), title='Window width' ,nticks=12, showticklabels=bool(j == 6),row=1, col=2).show(render='svg')
# figure.write_image(f'{images_path}scatter-rw-fig-wr.pdf')
#         print(f'{images_path}scatter-rw-fig-{j}-wr.pdf')
#     'MCPR-2021/images/sca'
tmp = df_multi_1.groupby(level=['model', 'offset']).agg(city_func)


## Minimuns

In [16]:
# link = 'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/rw-r.feather'
# df_multi_1 = pd.read_feather(link)
# df_multi_1.set_index(['model', 'function', 'offset', 'city'], inplace=True)
paths = (
    'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/sw-r.feather',
    'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/sw-wr.feather',
    'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/rw-r.feather',
    'https://raw.githubusercontent.com/HBaena/forecasting-criminal-incidence/main/rw-wr.feather',
)
indexs = (
    ['model', 'offset', 'city'],
    ['model', 'offset', 'city'],
    ['model', 'function', 'offset', 'city'],
    ['model', 'function', 'offset', 'city'],    
)
mins = [pd.read_feather(path).set_index(index).groupby(level='offset').min()['mean_absolute_error'] \
         for index, path in zip(indexs, paths)]

# df_multi = pd.read_csv('sw-r.csv', index_col=['model', 'offset', 'city'])
# tmp = df_multi.groupby(level=[0, 1]).mean()
# min_rw_wr = tmp.groupby(level='model').min()['mean_absolute_error']

mins = pd.concat(mins, 
    axis=1, 
    keys=[
        'SW with replacement',
        'SW without replacement',
        'RW with replacement',
        'RW without replacement',
    ]
)
# mins.to_latex(f'{images_path}mins.tex')
mins

,SW with replacement,SW without replacement,RW with replacement,RW without replacement
offset,,,,
1,0.069250,0.069250,0.069707,0.069707
2,0.069129,0.072955,0.068266,0.070309
3,0.061372,0.069395,0.066531,0.070130
4,0.069410,0.066560,0.067791,0.069723
5,0.065000,0.064593,0.069177,0.069443
6,0.077796,0.069230,0.069092,0.069850
7,0.073257,0.070063,0.068091,0.069072
8,0.066102,0.069328,0.067846,0.068520
9,0.069156,0.068752,0.069556,0.069301


In [19]:

mins = [pd.read_feather(path).set_index(index).groupby(level='offset').min()['mean_absolute_error'] \
         for index, path in zip(indexs, paths)]

mins = [pd.read_feather(path).set_index(index
                  ).pivot_table(index='model',columns='offset',aggfunc=min
                                )['mean_absolute_error'] \
         for index, path in zip(indexs, paths)]
mins = pd.concat(mins, 
    axis=0, levels='model'
).groupby(level=0).min()
mins

offset,1,2,3,4,5,6,7,8,9,10,11,12
model,,,,,,,,,,,,
BayesianRidge,0.072021,0.069117,0.069173,0.069939,0.070367,0.069177,0.069245,0.069361,0.069665,0.069369,0.069091,0.069064
LinearRegression,0.071594,0.072955,0.069395,0.069797,0.071549,0.070597,0.070212,0.070448,0.071758,0.073995,0.071838,0.070375
MLPRegressor,0.082380,0.068266,0.061372,0.066560,0.064593,0.070493,0.068091,0.066102,0.069633,0.067570,0.063467,0.066071
Ridge,0.069707,0.070929,0.071275,0.069410,0.069177,0.069092,0.069391,0.069598,0.069156,0.069586,0.069311,0.069507
SGDRegressor,0.107694,0.080501,0.074197,0.069723,0.069443,0.071867,0.073173,0.071317,0.068752,0.071822,0.069772,0.071327
SVR,0.069250,0.069122,0.069431,0.069566,0.069126,0.069846,0.069072,0.069328,0.069376,0.069500,0.070073,0.070338


## Correlation

In [ ]:
def corr_lag(df:pd.DataFrame, serie:pd.Series, lag:iter, 
             absval:bool=False, sort:bool=True, func=lambda df, lag: df.shift(lag)):
    tmp = lambda df, serie, lag : func(df, lag).corrwith(serie)
    result = pd.concat([tmp(data, serie, i) for i in lag],
        axis=1)
    if absval:
        result = result.abs()
    if sort:
        return result.assign(m=result.sum(axis=1)).sort_values('m', ascending=False).drop('m', axis=1)
    return result 

serie = data.groupby(level='date').mean()[target]
tmp = corr_lag(
            data.groupby(level='date').mean(), 
            serie, range(0, 16), 
            sort=True, absval=False, 
              )
fig = heatmap(tmp.columns, tmp.index, tmp, 
              template=r'Crime: %{x}<br>Lagged months: %{y}<br>Corr: %{z}')
fig.update_layout(
#     legend=dict(title='Correlation<br>Absolute value'),
margin=dict(b=200),
xaxis=dict(title='<span style="font-weight: bold;">Crime</span>', 
           tickvals=tuple(range(len(tmp.index))), 
           ticktext=tuple(map(lambda item: shorten(item, 25), tmp.index))),
yaxis=dict(title='<span style="font-weight: bold;">Lag month</span>'),
title=title_style.format('Correlation between the amount of <b>murders</b> and other crimes by past months'),
).show(render='svg')

In [ ]:
plot_multivariate_timeserie(data, normalize=True).update_layout(
    title=title_style.format('Behavior of difference'),
        xaxis=dict(title='Date'),
        yaxis=dict(title='Diff')
                                                              )

## Data exploration

In [ ]:
go.Figure(go.Bar(x=tmp.index, y=tmp['HOMICIDIO DOLOSO'])).update_layout(xaxis=dict(rangeslider=dict(visible=True)))

In [ ]:
# Slinding window of size 1
tmp_shift = tmp.shift(1)
display(tmp_shift.head())
# Add target value to the shifted data frame
# add 
tmp_shift['Target'] = tmp['HOMICIDIO DOLOSO']
# Reoving the first row
tmp_shift = tmp_shift[1:]
# Split train and test samples
train = tmp_shift.loc[:'2018']
test = tmp_shift.loc['2019':'2020-may']
# Selecting features and target
target = tmp_shift.columns[tmp_shift.columns == 'Target']
features = tmp_shift.columns[~(tmp_shift.columns == 'Target')]

In [ ]:
# Attributes to variate
attr_activation = ['identity', 'logistic', 'tanh', 'relu',]
attr_solver = ('lbfgs', 'sgd', 'adam')
attr_learning_rate = ('constant', 'invscaling', 'adaptive')
attr_max_iter = (100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 20000, 30000, 40000, 50000)
attr_hidden_layers = (10, 20, 30, 40, 50, 100, 200, 300, 500, 1000, 5000, 10000)
metrics = (mean_absolute_error, mean_squared_error)
x_train = train[features].values
y_train = train[target[0]].to_numpy()
x_test = test[features].values
y_test = test[target[0]].to_numpy()

In [ ]:
def meassure_performance(
                            model_func, metrics, attrs, attr_name,
                            x_train, y_train, x_test, y_test,
                            constant_attrs=dict()
                        ):
    results = pd.DataFrame(
        index=attrs, # Set the range of attributes as index
        columns=list(map(lambda a:a.__name__, metrics)) + ['time'] # metrics + time
    )
    for attr in attrs:
        constant_attrs[attr_name] = attr # add such attribute to te attributes dict
        print(constant_attrs) # Print the current attribute value
        t1 = time.process_time_ns() # Meassure time before training
        model = model_func(
            **constant_attrs # Make model with such attrs
        ).fit(x_train, y_train) # Train model inline
        t2 = time.process_time_ns() # Meassure time after training
        predictions = model.predict(x_test) # Predict with the test data 
        for metric in metrics:
            results[metric.__name__][attr] = metric(y_test, predictions) # Verify performance
                                                                         # with each metric
        results['time'][attr] = (t2 - t1)*1e-9                           # Add time excecutinon
        print(f'Time: {t2 - t1}')
        del model
    return results
def dict_of_df_to_multiindex_df(dictionary, axis=1):
    return pd.concat(dictionary.values(), keys=dictionary.keys(), axis=axis)


In [ ]:
results = dict()
for activation in attr_activation: # iterate over a 
    print("ACTIVATION: ", activation)
    results[activation] = meassure_performance(neural_network.MLPRegressor, 
                                    metrics, attr_hidden_layers, 
                                   'hidden_layer_sizes', x_train, y_train, x_test, y_test,
                                    {'activation' : activation}
                                    )

In [ ]:
results = dict_of_df_to_multiindex_df(results)

In [ ]:
# write
results.to_csv('results-activation function-hidden layers.csv')

In [ ]:
temp = pd.read_csv('results-activation function-hidden layers.csv', header=[0,1], index_col=[0])

In [ ]:
cols = ('mean_absolute_error', 'mean_squared_error')
for activation in attr_activation:
    scatter = list()
    box = list()
    fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], subplot_titles=['Lines plot', 'Boxplot'])
    for i, col in enumerate(cols):
        scatter.append(
            go.Scatter(
                x=results[activation].index,
                y=results[activation][col],
                name='<span style="font-size=8px;">' + col.replace("_", ' ').title() + ' - Line</span>',
                marker=dict(color=colors[i]),
                legendgroup=col))
        box.append(
            go.Box(y=results[activation][col],
            name='<span style="font-size=8px;">' + col.replace("_", ' ').title() + ' - Box</span>', 
            marker=dict(color=colors[i]), legendgroup=col))
    for plot in scatter:
        fig.add_trace(plot, row=1, col=1)
    fig.update_xaxes(rangeslider=dict(visible=True), type='log',
                     title='Hidden layers',
                    row=1, col=1)    
    for plot in box:
        fig.add_trace(plot, row=1, col=2)
    fig.update_xaxes(showticklabels=True, title='Metric',
                     tickvals=(0, 1), 
                     ticktext=list(map(lambda col: tick_style.format(col.replace("_", ' ').title()) ,cols)),
                    row=1, col=2)    
    fig.update_layout(title=title_style.format(f'Model\'s error for <b>{activation}</b> activation function'), 
                      font=dict(family='roboto, monospace'), legend=dict(title='Metric')).show()
    del fig

In [ ]:
fig = go.Figure()
for i, activation in enumerate(attr_activation):
    for j, col in enumerate(cols):
        fig.add_trace(
                go.Scatter(
                    x=results[activation].index,
                    y=results[activation][col],
                    name='<span style="font-size=8px;">' + activation + '</span>',
                    marker=dict(color=colors[i]),
                    hovertext=col.replace('_', ' ').title(),
                    legendgroup=activation,
                    showlegend=bool(j)
                )
    )
fig.update_xaxes(type='log', title='Hidden layers', rangeslider=dict(visible=True)).update_yaxes(title='Error')
fig.update_layout(title=title_style.format('Model\'s error according the activation function'), 
                  legend=dict(title='Activation function')).show()

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Mean absolute error', 'Mean squared error'])
absolute = list()
squared = list()
for i, activation in enumerate(attr_activation):
    for j, col in enumerate(cols):
        temp =  go.Scatter(
                    x=results[activation].index,
                    y=results[activation][col],
                    name='<span style="font-size=8px;">' + activation + ' - ' + col.replace('_', ' ').title() + '</span>',
                    marker=dict(color=colors[i]),
#                     hovertext=col.replace('_', ' ').title(),
                    legendgroup=activation,
                )

        if j:
            squared.append(temp)
        else:
            absolute.append(temp)
for plot in absolute:
    fig.add_trace(plot, col=1, row=1)
for plot in squared:
    fig.add_trace(plot, col=1, row=2)
fig.update_xaxes(type='log', title='',col=1, row=1).update_xaxes(type='log', rangeslider=dict(visible=True), 
                                                                 title='Number of hidden layers',col=1, row=2)
fig.update_layout(title=title_style.format('Model\'s error according to the activation function and the number of hidden layers')).show()    

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=['Mean absolute error', 'Mean squared error'])
absolute = list()
squared = list()
for i, activation in enumerate(attr_activation):
    for j, col in enumerate(cols):
        temp =  go.Box(
                    y=results[activation][col],
                    name='<span style="font-size=8px;">' + activation + ' - ' + col.replace('_', ' ').title() + '</span>',
                    marker=dict(color=colors[i]),
#                     hovertext=col.replace('_', ' ').title(),
                    legendgroup=activation,
                )

        if j:
            squared.append(temp)
        else:
            absolute.append(temp)
for plot in absolute:
    fig.add_trace(plot, col=1, row=1)
for plot in squared:
    fig.add_trace(plot, col=2, row=1)
fig.update_xaxes(type='log', title='',col=1, row=1).update_xaxes(type='log',title='',col=2, row=1)
fig.update_layout(title=title_style.format('Model\'s error according to the activation function and the number of hidden layers')).show()    

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True, subplot_titles=['Lines plot', 'Boxplot'])
scatter = list()
box = list()
for i, activation in enumerate(attr_activation):
        scatter.append(go.Scatter(
            x=results[activation].index,
            y=results[activation]['time'],
            name=f'<span style="font-size=8px;"> {activation} - Line</span>',
            marker=dict(color=colors[i]),
            legendgroup=activation
        ))
        box.append(go.Box(
            y=results[activation]['time'],
            name=f'<span style="font-size=8px;"> {activation} - Line</span>',
            marker=dict(color=colors[i]),
            legendgroup=activation
        ))

for plot in scatter:
    fig.add_trace(plot, row=1, col=1)
fig.update_xaxes(
            rangeslider=dict(visible=True), type='log', 
            title='Hidden layers',
            row=1, col=1).update_yaxes(
            title='Seconds',
            row=1, col=1)    
for plot in box:
    fig.add_trace(plot, row=1, col=2)
fig.update_xaxes(
            title='Activation function',
            row=1, col=2).update_yaxes(
            visible=False,
            row=1, col=2).update_xaxes(tickvals=list(range(len(attr_activation))), 
                                       ticktext=attr_activation, row=1, col=2)

fig.update_layout(
#                 xaxis=dict(type='log', title='hidden layers'), 
#                 yaxis=dict(title='Execution Seconds'),
                legend=dict(title=subtitle_style.format('Activation function')),
                title=title_style.format('Execution time for model\'s training according the activation function and hidden layers')
)

In [ ]:
results_2 = dict()
attrs = {'activation' :'relu'}
 
for attr in attr_max_iter: # iterate over a 
    print(attr)
    attrs['max_iter'] = attr
    results_2[attr] = meassure_performance(
        neural_network.MLPRegressor, 
        metrics, attr_hidden_layers, 
        'hidden_layer_sizes', x_train, y_train, x_test, y_test,
        constant_attrs=attrs
    )

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Mean absolute error', 'Mean squared error'])
absolute = list()
squared = list()
for i, max_iter in enumerate(attr_max_iter):
    for j, col in enumerate(cols):
        temp =  go.Scatter(
                    x=results_2[max_iter].index,
                    y=results_2[max_iter][col],
                    name='<span style="font-size=8px;">' + str(max_iter) + ' - ' + col.replace('_', ' ').title() + '</span>',
                    marker=dict(color=colors[i]),
#                     hovertext=col.replace('_', ' ').title(),
                    legendgroup=str(max_iter),
                )

        if j:
            squared.append(temp)
        else:
            absolute.append(temp)
for plot in absolute:
    fig.add_trace(plot, col=1, row=1)
for plot in squared:
    fig.add_trace(plot, col=1, row=2)
fig.update_xaxes(type='log', title='',col=1, row=1).update_xaxes(type='log', rangeslider=dict(visible=True), 
                                                                 title='Number of hidden layers',col=1, row=2)
fig.update_layout(title=title_style.format('Model\'s error according to the max_iter function and the number of hidden layers')).show()    

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Mean absolute error', 'Mean squared error'])
absolute = list()
squared = list()
for i, max_iter in enumerate(attr_max_iter):
    for j, col in enumerate(cols):
        temp =  go.Box(
                    y=results_2[max_iter][col],
                    name='<span style="font-size=8px;">' + str(max_iter) + ' max iter </span>',
                    marker=dict(color=colors[i]),
#                     hovertext=col.replace('_', ' ').title(),
                    legendgroup=str(max_iter),
                    showlegend=bool(j)
                )

        if j:
            squared.append(temp)
        else:
            absolute.append(temp)
for plot in absolute:
    fig.add_trace(plot, col=1, row=1)
for plot in squared:
    fig.add_trace(plot, col=1, row=2)
fig.update_xaxes(title='Number of iteration',col=1, row=2, 
                tickvals=list(range(len(attr_max_iter))), 
                ticktext=list(map(str, attr_max_iter)), rangeslider=dict(visible=True))
fig.update_xaxes(title='',col=1, row=1, 
                tickvals=list(range(len(attr_max_iter))), 
                ticktext=list(map(str, attr_max_iter)))
fig.update_layout(title=title_style.format('Model\'s error according to the max_iter and the number of hidden layers')).show()    

In [ ]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True, subplot_titles=['Lines plot', 'Boxplot'])
scatter = list()
box = list()
for i, max_iter in enumerate(attr_max_iter):
        scatter.append(go.Scatter(
            x=results_2[max_iter].index,
            y=results_2[max_iter]['time'],
            name=f'<span style="font-size=8px;"> {max_iter} - Line</span>',
            marker=dict(color=colors[i]),
            legendgroup=max_iter
        ))
        box.append(go.Box(
            y=results_2[max_iter]['time'],
            name=f'<span style="font-size=8px;"> {max_iter} - Line</span>',
            marker=dict(color=colors[i]),
            legendgroup=max_iter
        ))

for plot in scatter:
    fig.add_trace(plot, row=1, col=1)
fig.update_xaxes(
            rangeslider=dict(visible=True), type='log', 
            title='Hidden layers',
            row=1, col=1).update_yaxes(
            title='Seconds',
            row=1, col=1)    
for plot in box:
    fig.add_trace(plot, row=1, col=2)
fig.update_xaxes(
            title='Activation function',
            row=1, col=2).update_yaxes(
            visible=False,
            row=1, col=2).update_xaxes(tickvals=list(range(len(attr_max_iter))), 
                                       ticktext=attr_max_iter, row=1, col=2)

fig.update_layout(
#                 xaxis=dict(type='log', title='hidden layers'), 
#                 yaxis=dict(title='Execution Seconds'),
                legend=dict(title=subtitle_style.format('Activation function')),
                title=title_style.format('Execution time for model\'s training according the max_iter function and hidden layers')
)

In [ ]:
figure = go.Figure()
figure.add_trace(
    go.Scatter(x=results.index, y=results.Target, name='True values')
)
figure.add_trace(
    go.Scatter(x=results.index, y=results.Prediction, name='Predicted values')
)
figure.update_layout(xaxis=dict(rangeslider=dict(visible=True)))

In [ ]:
tmp_shift_2 = tmp.shift(2)

In [ ]:
pd.concat([tmp_shift, tmp_shift_2], axis="columns")